In [1]:
# import required packages
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense,BatchNormalization, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator

In [2]:
# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [3]:
# Preprocess all test images
train_generator = train_data_gen.flow_from_directory('Data/train',target_size=(48, 48),batch_size=64,color_mode="grayscale",class_mode='categorical')

# Preprocess all train images
validation_generator = validation_data_gen.flow_from_directory('Data/test',target_size=(48, 48),batch_size=64,color_mode="grayscale",class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [4]:
# create model structure
emotion_model = Sequential()

""" 1st convolution layer """
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(BatchNormalization())
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

""" 2nd convolution layer """
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(BatchNormalization())
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

""" 3rd convolution layer """
emotion_model.add(Conv2D(128, (3, 3), activation='relu'))
emotion_model.add(Conv2D(128, (3, 3), activation='relu'))
emotion_model.add(BatchNormalization())
emotion_model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

In [5]:
emotion_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the neural network/model
emotion_model_info = emotion_model.fit_generator(train_generator,steps_per_epoch=28709 // 64,epochs=50,validation_data=validation_generator,validation_steps=7178 // 64)


# save model structure in jason file
model_json = emotion_model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# save trained model weight in .h5 file
emotion_model.save_weights('model_weight.h5')

# Evaluate Model
score = emotion_model.evaluate_generator(validation_generator, steps=7067 // 64) 
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

<ipython-input-5-01c7c7f18eaf>:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(train_generator,steps_per_epoch=28709 // 64,epochs=50,validation_data=validation_generator,validation_steps=7178 // 64)


Epoch 1/50
448/448 [==============================] - 337s 749ms/step - loss: 1.7579 - accuracy: 0.3226 - val_loss: 1.6216 - val_accuracy: 0.3542
Epoch 2/50
448/448 [==============================] - 258s 576ms/step - loss: 1.4770 - accuracy: 0.4273 - val_loss: 1.5048 - val_accuracy: 0.4152
Epoch 3/50
448/448 [==============================] - 259s 579ms/step - loss: 1.3484 - accuracy: 0.4847 - val_loss: 1.4892 - val_accuracy: 0.4442
Epoch 4/50
448/448 [==============================] - 261s 583ms/step - loss: 1.2705 - accuracy: 0.5162 - val_loss: 1.5264 - val_accuracy: 0.4558
Epoch 5/50
448/448 [==============================] - 256s 572ms/step - loss: 1.2086 - accuracy: 0.5418 - val_loss: 1.2265 - val_accuracy: 0.5332
Epoch 6/50
448/448 [==============================] - 262s 586ms/step - loss: 1.1600 - accuracy: 0.5589 - val_loss: 1.1887 - val_accuracy: 0.5552
Epoch 7/50
448/448 [==============================] - 269s 601ms/step - loss: 1.1091 - accuracy: 0.5809 - val_loss: 1.3766 -

<ipython-input-5-01c7c7f18eaf>:16: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  score = emotion_model.evaluate_generator(validation_generator, steps=7067 // 64)


Test loss:  2.273135185241699
Test accuracy:  0.5973011255264282
